In [1]:
import gensim
import numpy as np
import pandas as pd
import jieba
import os

# 导入数据
folder_path = 'danmu_text'
# 导入 word to vector 模型
W2V_model_path = './W2V_model/model_from_zh.model'
W2V_model = gensim.models.Word2Vec.load(W2V_model_path)
# word to vector 的字典可以用来检测库中有没有对应的词向量
W2V_dic = W2V_model.wv.index_to_key
# dim 是词向量的维度
word_vec_dim = len(W2V_model.wv[W2V_dic[1]])

print("dim =", word_vec_dim)

dim = 70


In [2]:
print('人口' in W2V_dic)
print('傻瓜' in W2V_dic)
print('炸药' in W2V_dic)
print('西瓜刀' in W2V_dic)
# 下面这个输出就是 '人口' 对应的词向量
print(W2V_model.wv['人口'])


True
True
True
True
[ 0.1383596  -0.17468986  0.26738378  0.27664915  0.7140228   0.30336457
 -0.32623005  0.14275849  0.20718439 -1.3750342  -0.15256792  0.5628893
 -0.12664007 -1.671796   -0.41980577  0.16698277  0.8566149   1.6146973
  1.32098     0.518146   -0.6968868   1.0875909   0.35259685 -1.1078038
 -0.06966351  0.65688205 -0.34248763  0.8551295  -1.3682519  -0.6258048
  0.03248164  0.35811904 -0.02586001  1.4380642  -0.97641194  0.8970954
 -0.7824455  -0.24037059 -0.11944317 -0.8639166   0.3388526  -0.25791183
 -1.2384986   0.15366787 -0.5133913  -0.571598    0.5292653  -0.41370398
  0.01003916  0.42355835 -0.41143772 -1.1394283   0.79049635 -0.3699302
  0.63536614 -0.12875251 -0.21281405  0.32406306 -0.33010438  0.4743404
 -2.3396661  -0.04518685 -0.9380626  -0.86265945  0.21989642 -0.7452054
  0.27972785 -0.09746973 -0.47074002 -0.53538364]


In [3]:
# 限制一条弹幕的最大词数，过长会被截断
max_len = 15

splited_barrages = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)

        # 读取txt文件内容
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        # 先把整个文件分成一条一条的弹幕
        barrages = content.split('\n')
        
        # 再用jieba.lcut把每条弹幕都分成词(str)组成的list
        for barr in barrages:
            splited_barrages.append(jieba.lcut(barr))

        print(splited_barrages)

input_data_len = len(splited_barrages)
print("data length =", input_data_len)
for i in range(input_data_len):
    # 限制词数
    splited_barrages[i] = splited_barrages[i][:max_len]
    input_data_i_len = len(splited_barrages[i])

    # 把一条弹幕的词逐个转化为词向量
    for j in range(input_data_i_len):
        # 找得到直接转
        if(splited_barrages[i][j] in W2V_dic):
            splited_barrages[i][j] = W2V_model.wv[splited_barrages[i][j]]
        # 找不到设为空
        elif(splited_barrages[i][j] not in W2V_dic):
            splited_barrages[i][j] = np.array(list(0 for k in range(word_vec_dim)))
        # 可以暂时忽略
        else:
            wv = W2V_model.wv[splited_barrages[i][j]]
            W2V_dic[splited_barrages[i][j]] = wv
            splited_barrages[i][j] = wv
    
    # 输出进度
    if(i % int(input_data_len / 10) == 0 and i != 0):
        print("\r" + str(i) + " sentences done", end = " ")

# 示例输出，这里可以看到这条弹幕的每个词都被转化成词向量(array)
print(splited_barrages[3])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\W\AppData\Local\Temp\jieba.cache
Loading model cost 0.888 seconds.
Prefix dict has been built successfully.


[['content', ',', 'target'], ['沉默', '，', '沉默', '，', '整天', '就', '知道', '沉默', '！', ',', '1'], ['哈哈哈哈', '哈哈哈', '这个', '马赛克', ' ', '谢谢', ',', '1'], ['勾手', '投篮', ',', '0'], ['？', '？', '？', '？', '？', '？', '？', '？', '？', '？', '？', '？', '？', '？', '？', '？', '你', '再说', '一遍', '？', '？', '？', ',', '-', '1'], ['?', ',', '0'], ['那', '也', '有', '可能', '是', '受益人', ',', '0'], ['《', '学', ' ', '而', ' ', '不', ' ', '玩', ' ', '则', ' ', '呆', '》', ',', '1'], ['麦克雷', ',', '0'], ['可能', '为了', '触发', '残血', 'buff', '吧', ',', '0'], ['哪个', '正常', '国家', '会', '有', '如此', '多', '情报部门', '?', ',', '-', '1'], ['草', ',', '1'], ['《', '福到', '了', '》', ',', '1'], ['我来', '了', '猫猫', '！', ',', '1'], ['这个', '坎', '不好过', '的', ',', '0'], ['怪核', ',', '0'], ['尼', '䧞', '怎么', '还', '没似', ',', '-', '1'], ['★', '★', '★', '☆', '☆', ',', '0'], ['谢谢', '罗', '老师', ',', '1'], ['这', '不', '玩游戏', '，', '开始', '讲', '人情世故', '了', ',', '0'], ['哈哈哈哈', ',', '1'], ['说', '宋江', '心里', '乐', '开花', '的', '，', '你', '是不是', '什么', '都', '没', '看', ',', '-', '1'], ['法律', '的', '最终'